In [ ]:
# 1. INSTALL
import os
import warnings
warnings.filterwarnings("ignore")

# NOTE: Internet access is required here for:
# 1. Installing external libraries (CLIP, ultralytics, thop).
# 2. Downloading the YOLOv5s model weights (via torch.hub in Cell 3).
# 3. Downloading the CLIP ResNet101 model weights (via clip.load in Cell 3).
print("⚙️ Installing Dependencies...")
os.system("pip install git+https://github.com/openai/CLIP.git -q")
os.system("pip install -U seaborn thop ultralytics scikit-learn -q")

# 2. IMPORTS
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
from sklearn.metrics import pairwise_distances
import clip

In [ ]:
class CFG:
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Dataset Paths
    DATASET_ROOT = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset'
    TRAIN_FOLDER = 'training_videos'
    TEST_FOLDER = 'testing_videos'
    
    # Hyperparameters
    CLIP_ARCH = "RN101"        
    YOLO_MODEL = 'yolov5s'     # Using Small model (Stable & Fast)
    
    IMG_SIZE = (224, 224)
    CORESET_RATIO = 0.01       # 1% Memory Retention
    
    # Ensemble & Fusion
    SEEDS = [42, 123, 2024]    
    W_LOCAL = 0.7              
    W_GLOBAL = 0.3             
    SMOOTHING_SIGMA = 17       
    CONF_THRES = 0.5           
    
    WORK_DIR = './working_dir'

os.makedirs(CFG.WORK_DIR, exist_ok=True)

In [ ]:
# --- Object Detector (YOLOv5) ---
class PersonDetector:
    def __init__(self, device):
        print(f"   > Loading {CFG.YOLO_MODEL}...")
        self.model = torch.hub.load('ultralytics/yolov5', CFG.YOLO_MODEL, pretrained=True, verbose=False)
        self.model.to(device)
        self.model.classes = [0] # Class 0 = Person
        self.model.conf = CFG.CONF_THRES

    def detect_and_crop(self, img_path):
        results = self.model(img_path)
        crops = []
        if len(results.xyxy[0]) > 0:
            img = Image.open(img_path).convert('RGB')
            for *box, conf, cls in results.xyxy[0]:
                x1, y1, x2, y2 = map(int, box)
                c = img.crop((x1, y1, x2, y2)).resize(CFG.IMG_SIZE, Image.Resampling.BILINEAR)
                crops.append(c)
        return crops

# --- Feature Extractor (CLIP) ---
class VideoPatchCore(nn.Module):
    def __init__(self, device):
        super().__init__()
        print(f"   > Loading CLIP {CFG.CLIP_ARCH}...")
        self.model, self.preprocess = clip.load(CFG.CLIP_ARCH, device=device)
        self.model.eval()
        self.features = {}
        # Hook into intermediate layers
        self.model.visual.layer2.register_forward_hook(self.hook('layer2'))
        self.model.visual.layer3.register_forward_hook(self.hook('layer3'))

    def hook(self, name):
        def fn(model, input, output): self.features[name] = output
        return fn

    def embed(self, pil_images):
        if not pil_images: return None
        batch = torch.stack([self.preprocess(img) for img in pil_images]).to(CFG.DEVICE)
        
        with torch.no_grad():
            self.model.visual(batch)
        
        l2, l3 = self.features['layer2'], self.features['layer3']
        if l2.shape[-2:] != l3.shape[-2:]:
            l2 = F.interpolate(l2, size=l3.shape[-2:], mode='bilinear', align_corners=False)
        
        # Concatenate and Average Pool to 1x1 vector (No Patching)
        embedding = torch.cat([l2, l3], dim=1)
        vector = F.adaptive_avg_pool2d(embedding, 1).flatten(1)
        return vector.cpu()

# --- Memory Optimization (Coreset) ---
def subsample(features, ratio=0.01, seed=42):
    np.random.seed(seed)
    
    if len(features) > 20000:
        indices = np.random.choice(len(features), 20000, replace=False)
        features = features[indices]

    num_samples = max(1, int(len(features) * ratio))
    features_np = features.float().numpy() 
    
    # Greedy K-Center Selection
    current_idx = np.random.choice(len(features_np))
    selected = [current_idx]
    min_dists = pairwise_distances(features_np[current_idx:current_idx+1], features_np).flatten()
    
    for _ in range(1, num_samples):
        new_idx = np.argmax(min_dists)
        selected.append(new_idx)
        new_dists = pairwise_distances(features_np[new_idx:new_idx+1], features_np).flatten()
        min_dists = np.minimum(min_dists, new_dists)
        
    return features[selected].to(CFG.DEVICE)

In [ ]:
def run_pipeline():
    detector = PersonDetector(CFG.DEVICE)
    vpc = VideoPatchCore(CFG.DEVICE)
    
    # ---------------------------------------------------------
    # STEP 1: EXTRACT FEATURES (Memorization)
    # ---------------------------------------------------------
    print("\n[PHASE 1] Extracting Features from Training Data...")
    train_path = os.path.join(CFG.DATASET_ROOT, CFG.TRAIN_FOLDER)
    video_folders = sorted(glob.glob(os.path.join(train_path, '*')))
    
    local_cache = []
    global_cache = []
    
    print(f"   > Scanning {len(video_folders)} videos...")
    for video_dir in tqdm(video_folders):
        frames = sorted(glob.glob(os.path.join(video_dir, '*.jpg')))
        
        # Subsample every 5th frame for speed
        for frame_path in frames[::5]: 
            # Global Feature (Whole Frame)
            img = Image.open(frame_path).convert('RGB')
            global_img = img.resize(CFG.IMG_SIZE, Image.Resampling.BILINEAR)
            global_cache.append(vpc.embed([global_img]))
            
            # Local Features (People Crops)
            crops = detector.detect_and_crop(frame_path)
            if crops:
                local_cache.append(vpc.embed(crops))

    if not local_cache:
        raise ValueError("CRITICAL: No people detected in training data.")

    full_local = torch.cat(local_cache, dim=0)
    full_global = torch.cat(global_cache, dim=0)
    print(f"   > Extracted: {len(full_local)} Local vectors, {len(full_global)} Global vectors.")

    # ---------------------------------------------------------
    # STEP 2: ENSEMBLE INFERENCE (3 Seeds)
    # ---------------------------------------------------------
    final_preds = {} 
    
    print("\n[PHASE 2] Starting Ensemble Inference...")
    test_path = os.path.join(CFG.DATASET_ROOT, CFG.TEST_FOLDER)
    test_videos = sorted(glob.glob(os.path.join(test_path, '*')))
    
    # Pre-load all test frame paths
    all_test_frames = []
    for video_dir in test_videos:
        vid_id = os.path.basename(video_dir)
        frames = sorted(glob.glob(os.path.join(video_dir, '*.jpg')))
        for f in frames:
            all_test_frames.append((vid_id, f))
            # Init prediction storage
            fname = os.path.basename(f)
            try: fid = f"{int(vid_id)}_{int(fname.split('_')[1].split('.')[0])}"
            except: fid = f"{vid_id}_{fname}"
            final_preds[fid] = 0.0

    # Ensemble Loop
    for i, seed in enumerate(CFG.SEEDS):
        print(f"\n--- RUN {i+1}/{len(CFG.SEEDS)} (Seed {seed}) ---")
        
        # Build specific memory bank for this seed
        local_bank = subsample(full_local, CFG.CORESET_RATIO, seed=seed)
        global_bank = subsample(full_global, CFG.CORESET_RATIO, seed=seed)
        
        # Score Test Data
        print(f"   > Scoring...")
        for vid_id, frame_path in tqdm(all_test_frames):
            fname = os.path.basename(frame_path)
            try: fid = f"{int(vid_id)}_{int(fname.split('_')[1].split('.')[0])}"
            except: fid = f"{vid_id}_{fname}"
                
            # Global Score (Nearest Neighbor)
            img = Image.open(frame_path).convert('RGB')
            global_img = img.resize(CFG.IMG_SIZE, Image.Resampling.BILINEAR)
            g_vec = vpc.embed([global_img]).to(CFG.DEVICE)
            d_global = torch.cdist(g_vec.float(), global_bank.float()).min(dim=1)[0].item()
            
            # Local Score (Max anomaly of any person in frame)
            crops = detector.detect_and_crop(frame_path)
            if len(crops) == 0: 
                d_local = 0.0
            else:
                l_vecs = vpc.embed(crops).to(CFG.DEVICE)
                d_local = torch.cdist(l_vecs.float(), local_bank.float()).min(dim=1)[0].max().item()
            
            # Weighted Fusion
            score = (CFG.W_LOCAL * d_local) + (CFG.W_GLOBAL * d_global)
            final_preds[fid] += score

    # Average scores across seeds
    for k in final_preds: 
        final_preds[k] /= len(CFG.SEEDS)
        
    return final_preds

In [ ]:
if __name__ == "__main__":
    results = run_pipeline()
    
    print("\n[PHASE 3] 📝 Generating Submission...")
    df = pd.DataFrame(list(results.items()), columns=['Id', 'Raw_Score'])
    
    # 1. Robust Sorting (Required for smoothing)
    try:
        df['VidID'] = df['Id'].apply(lambda x: int(x.split('_')[0]))
        df['FrameID'] = df['Id'].apply(lambda x: int(x.split('_')[1]))
        df = df.sort_values(['VidID', 'FrameID'])
    except:
        print("⚠️ Warning: Non-standard ID format. Sorting alphanumerically.")
        df = df.sort_values('Id')
    
    # 2. Gaussian Smoothing
    df['Predicted'] = gaussian_filter1d(df['Raw_Score'], sigma=CFG.SMOOTHING_SIGMA)
    
    # 3. Min-Max Normalization
    _min, _max = df['Predicted'].min(), df['Predicted'].max()
    if _max > _min:
        df['Predicted'] = (df['Predicted'] - _min) / (_max - _min)
    else:
        df['Predicted'] = 0.0
        
    # 4. Save Final CSV
    submission = df[['Id', 'Predicted']]
    submission.to_csv('submission.csv', index=False)
    
    print("✅ DONE. Submission 'submission.csv' ready.")
    print(submission.head())